In [4]:
!pip install torch
!pip install haystack-ai#==2.2.4
!pip install haystack-experimental#==0.1.0
!pip install sentence-transformers#==3.0.1
!pip install transformers#==4.42.3
!pip install ollama-haystack

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [5]:
import transformers
import torch

from haystack_integrations. components.generators.ollama import OllamaGenerator

In [10]:
generator = OllamaGenerator(model="llama3.1",
  url = "http://103.220.68.101:11434/api/generate",
  generation_kwargs={
    "num_predict": 100,
    "temperature": 0.9,
  })

In [11]:
print(generator. run("Who is the best American actor?"))

{'replies': ['A question that\'s sure to spark some debate!\n\nChoosing the "best" American actor is subjective and often depends on personal opinions, preferences, and criteria for evaluation. Here are a few possible approaches:\n\n1. **Critical acclaim**: Consider actors who have received numerous awards and nominations from reputable organizations like the Academy Awards (Oscars), Golden Globes, BAFTAs, and Critics\' Choice Movie Awards.\n2. **Box office success**: Look at actors who have consistently delivered hits at the box'], 'meta': [{'model': 'llama3.1', 'created_at': '2024-08-26T02:33:25.3260738Z', 'done': True, 'done_reason': 'length', 'context': [128009, 128006, 882, 128007, 271, 15546, 374, 279, 1888, 3778, 12360, 30, 128009, 128006, 78191, 128007, 271, 32, 3488, 430, 596, 2771, 311, 15541, 1063, 11249, 2268, 96144, 279, 330, 16241, 1, 3778, 12360, 374, 44122, 323, 3629, 14117, 389, 4443, 18463, 11, 19882, 11, 323, 13186, 369, 16865, 13, 5810, 527, 264, 2478, 3284, 20414, 

In [12]:
from haystack_integrations. components.generators.ollama import OllamaGenerator
from haystack import Pipeline, Document
from haystack. components. retrievers. in_memory import InMemoryBM25Retriever
from haystack. components. builders. prompt_builder import PromptBuilder
from haystack. document_stores. in_memory import InMemoryDocumentStore

In [13]:
template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
{{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

In [25]:
docstore = InMemoryDocumentStore()
# docstore.write_documents([Document(content="I really like summer"),
#                           Document (content="My favorite sport is soccer"),
#                           Document (content="I don't like reading sci-fi books"),
#                           Document (content="I don't like crowded places") , ])

docstore.write_documents([Document(content="Tôi thích mùa hè"),
                          Document(content="Môn thể thao yêu thích của tôi là bóng đá"),
                          Document(content="Tôi không thích đọc sách khoa học viễn tưởng"),
                          Document(content="Tôi không thích những nơi đông người") , ])

4

In [26]:
generator = OllamaGenerator(model="llama3.1",
                            url = "http://103.220.68.101:11434/api/generate",
                            generation_kwargs={
                              "num_predict": 100,
                              "temperature": 0.9,
                            })

In [27]:
pipe = Pipeline()
pipe.add_component("retriever", InMemoryBM25Retriever(document_store=docstore))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", generator)
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [28]:
#query="what is my favorite sport?"
query="Bạn yêu thích môn thể thao nào?"
result = pipe.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

In [29]:
print(result)

{'llm': {'replies': ['Câu trả lời phù hợp nhất cho câu hỏi này sẽ là:\n\nBóng đá!\n\nLý do là trong thông tin đã cho, đã được đề cập rằng "Môn thể thao yêu thích của tôi là bóng đá".'], 'meta': [{'model': 'llama3.1', 'created_at': '2024-08-26T02:46:00.4282917Z', 'done': True, 'done_reason': 'stop', 'context': [128009, 128006, 882, 128007, 1432, 22818, 279, 2768, 2038, 11, 4320, 279, 3488, 382, 2014, 1473, 44, 62552, 100540, 116993, 103755, 104672, 60835, 102598, 39015, 109385, 108972, 271, 127806, 104672, 112199, 120221, 271, 127806, 54137, 104672, 100557, 105201, 109708, 100524, 271, 127806, 54137, 104672, 113354, 96182, 102637, 100705, 3355, 5080, 101792, 108546, 1432, 14924, 25, 112377, 103755, 104672, 107551, 100540, 116993, 102176, 7801, 128009, 128006, 78191, 128007, 271, 34, 101208, 106595, 106866, 105091, 100827, 101071, 2665, 106821, 107193, 97635, 100770, 39015, 1473, 33, 102818, 108972, 2268, 43, 20195, 656, 39015, 70186, 93530, 25826, 80226, 2665, 11, 80226, 64578, 101218, 